# Pandemic Registry Key
Detects Pandemic Windows Implant

## Rule Content
```
- action: global
  title: Pandemic Registry Key
  id: 47e0852a-cf81-4494-a8e6-31864f8c86ed
  status: experimental
  description: Detects Pandemic Windows Implant
  references:
  - https://wikileaks.org/vault7/#Pandemic
  - https://twitter.com/MalwareJake/status/870349480356454401
  tags:
  - attack.lateral_movement
  - attack.t1105
  author: Florian Roth
  detection:
    condition: 1 of them
  fields:
  - EventID
  - CommandLine
  - ParentCommandLine
  - Image
  - User
  - TargetObject
  falsepositives:
  - unknown
  level: critical
- logsource:
    product: windows
    service: sysmon
  detection:
    selection1:
      EventID: 13
      TargetObject:
      - \REGISTRY\MACHINE\SYSTEM\CurrentControlSet\services\null\Instance*
      - \REGISTRY\MACHINE\SYSTEM\ControlSet001\services\null\Instance*
      - \REGISTRY\MACHINE\SYSTEM\ControlSet002\services\null\Instance*
- logsource:
    category: process_creation
    product: windows
  detection:
    selection2:
      Command: loaddll -a *

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:(\\REGISTRY\\MACHINE\\SYSTEM\\CurrentControlSet\\services\\null\\Instance* OR \\REGISTRY\\MACHINE\\SYSTEM\\ControlSet001\\services\\null\\Instance* OR \\REGISTRY\\MACHINE\\SYSTEM\\ControlSet002\\services\\null\\Instance*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='Command.keyword:loaddll\ \-a\ *')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()